In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random
from collections import deque
from sys import maxsize

In [2]:
def createList(r1, r2):#option creation for random edge addition
    if (r1 == r2):
        return r1
    else:
        res = []
        while(r1 != r2):
            res.append(r1)
            r1 += 1
            if (r1 > 50):
                r1 = r1-50
        res.append(r2)
    return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
        self.paths = self.all_shortest_paths()
        
    def find_paths(self, paths, path, parent, end):
        if end == -1:
            paths.append(path.copy())
            return

        for par in parent[end]:
            path.append(end)
            self.find_paths(paths, path, parent, par)
            path.pop()
        
    def bfs_all_paths(self, parent, start):
    
        dist = [maxsize for _ in range(51)]
        q = deque()
        q.append(start)
        parent[start] = [-1]
        dist[start] = 0

        while q:
            u = q[0]
            q.popleft()
            for v in self.graph[u]:
                if dist[v] > dist[u] + 1:
                    dist[v] = dist[u] + 1
                    q.append(v)
                    parent[v].clear()
                    parent[v].append(u)

                elif dist[v] == dist[u] + 1:
                    parent[v].append(u)

    def all_shortest_paths(self):
        shortest_paths = dict()
        for i in range(1, 51):
            for j in range(1, 51):
                paths, path, shortest_path = [], [], []

                parent = [[] for _ in range(51)]
                self.bfs_all_paths(parent, i)

                self.find_paths(paths, path, parent, j)

                for move in paths:
                    move = list(reversed(move))
                    shortest_path.append(move)
                    
                shortest_paths[i,j] = shortest_path

        return shortest_paths

                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1): #deleting adjacent options so that double edges arent made
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else: 
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
    
    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self
        

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [7]:
class Utility:
    def __init__(self):
        self.Ustar = np.zeros((51, 51, 51)) # Initialize UStar map
    
    #determines the value of each move the agent can take to get to the prey
    def action_value(self, g, agentpos, preyneighbors, predneighbors, pdpos):
        future_utility = 0
        for preypos in preyneighbors:
            for predpos in predneighbors:
                # figure out distracted probability
                shortest_paths = g.paths[pdpos, agentpos]
                count = 0
                for path in shortest_paths:
                    if predpos in path:
                        count += 1
                #calculating the total probability
                probability = (1.0/len(preyneighbors))*((0.4/len(predneighbors)) + (0.6 * (count/len(shortest_paths))))
                future_utility += probability*(1 + self.Ustar[agentpos][preypos][predpos])
        return future_utility

    def value_iteration(self, g):
        
        #initialize UStar
        for agentpos in range(1, 51): 
            for preypos in range(1, 51):
                for predpos in range(1, 51):
                    if (agentpos == preypos) and (agentpos != predpos):
                        self.Ustar[agentpos][preypos][predpos] = 0
                    elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                        self.Ustar[agentpos][preypos][predpos] = 10e8
                    else:
                        self.Ustar[agentpos][preypos][predpos] = 1

        #Update UStar based on the likely movements of the prey and predator
        while True:
            currUstar = np.zeros((51, 51, 51))
            for agentpos in range(1, 51): 
                for preypos in range(1, 51):
                    for predpos in range(1, 51):
                        if (agentpos == preypos) and (agentpos != predpos):
                            currUstar[agentpos][preypos][predpos] = 0.0
                        elif (agentpos == predpos) or (abs(agentpos - predpos) == 1):
                            currUstar[agentpos][preypos][predpos] = 100000
                        elif (abs(agentpos - preypos) == 1):
                            currUstar[agentpos][preypos][predpos] = 1
                        else:
                            action_values = []
                            for move in g.graph[agentpos]:
                                # Run loop until value converges
                                preyneighbors = []
                                preyneighbors.append(preypos)
                                predneighbors = []
                                for neighbor in g.graph[preypos]:
                                    preyneighbors.append(neighbor)
                                for neighbor in g.graph[predpos]:
                                    predneighbors.append(neighbor)
                                currval = self.action_value(g, move, preyneighbors, predneighbors, predpos)
                                action_values.append(currval)           
                            currUstar[agentpos][preypos][predpos] = min(action_values) 
      
            #run until error is minimized
            max_error = 0
            for agentpos in range(1, 51): 
                for preypos in range(1, 51):
                    for predpos in range(1, 51):
                        max_error = max(max_error, abs(self.Ustar[agentpos][preypos][predpos] - currUstar[agentpos][preypos][predpos]))           
            if max_error < 0.001:
                break
            self.Ustar = currUstar
        

In [8]:
def AgentUStar(g, utility):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    while (fail == 0 and steps <= 100):
        
        #move agent to the neighbor with the lowest utility
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility.Ustar[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility.Ustar[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break

        #move prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        #move predator
        predator.pred_distracted_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [9]:
g = Graph()
graph = g.addEdge()
while graph:
    graph = g.addEdge()
g.paths = g.all_shortest_paths()
utility = Utility()
utility.value_iteration(g)

In [10]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 3000:
    
    res, path = AgentUStar(g, utility)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2987
Number of losses: 13
Number of timeouts: 0
Success Rate: 0.9956666666666667


In [11]:
def AgentUStarPartial(g, utility):
    agent_pos = random.randint(1,50)
    predator = Predator(agent_pos)
    prey = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    
    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of agent at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        prey_not = 1-belief_vector[agent_pos-1] #obviously prey isnt at agents position so get probability of prey not being in that position
        belief_vector[agent_pos-1] = 0 #making belief of prey in agents node 0
        for i in range(len(belief_vector)):
                belief_vector[i] = belief_vector[i]/prey_not #normalizing
    
    while (fail == 0 and steps <= 100):
        
        max_prob = max(belief_vector)
        
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the prey
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)
        possible_prey_pos = random.choice(list_of_positions_max_prob) #randomly selecting one of those positions
        
        best_node = agent_pos
        min_utility = 10000
        
        for action in g.graph[agent_pos]:
            if utility.Ustar[action][prey.pos][predator.pos] < min_utility:
                min_utility = utility.Ustar[action][prey.pos][predator.pos]
                best_node = action
        
        agent_pos = best_node
        path.append(agent_pos)
        steps += 1
        
        if (agent_pos == prey.pos): #if agent has already moved to location of prey
            break
        else: #if prey not caught, treat as survey and perform another belief update
            prey_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of prey not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/prey_not

        #move predator and prey
        prey.prey_move(g)

        if (prey.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator.pred_distracted_move(g, agent_pos)

        if(predator.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #making a temporary belief vector for correct calculations
        for i in range(len(belief_vector)):
            mag_sum =  temp_belief_vector[i]/(len(g.graph[i+1])+1) #probability of prey staying in same position
            i_neighbours = g.graph[i+1] #getting neighbours of current position
            #print(g.graph)
            #print(i+1)
            #print(i_neighbours)
            for bloke in i_neighbours:
                mag_sum += (temp_belief_vector[bloke-1]/(len(g.graph[bloke])+1)) #calculating probability of prey moving to node i if it was in node bloke
            belief_vector[i] = mag_sum #assigning probability
        
    #fail if timeout error    
    if(steps >= 100):
        fail = 2 

    return fail, path

In [12]:
wins = 0
losses = 0
timeout = 0
times = 0

while times < 3000:
    
    res, path = AgentUStarPartial(g, utility)
    
    if(res == 0):
        wins += 1
    elif(res == 1):
        losses += 1
    else:
        timeout += 1
    times += 1
    
print('Number of wins: ' + str(wins))
print('Number of losses: ' + str(losses))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2987
Number of losses: 13
Number of timeouts: 0
Success Rate: 0.9956666666666667
